In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import ipdb
import wandb
from wandb.keras import WandbCallback
print("Using tensorflow:",tf.__version__)

Using tensorflow: 2.4.1


In [33]:
wandb.init(project='Assignment 3', entity='iitm-cs6910-jan-may-2021-cs20m059-cs20m007')
wandb.run.name = "LSTM-Transliteration-"+wandb.run.id
wandb.run.save()

epoch,49
loss,0.01997
accuracy,0.99371
val_loss,0.2916
val_accuracy,0.93998
_runtime,578
_timestamp,1619797744
_step,49
best_val_loss,0.19149
best_epoch,11


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▅▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▃▅▆▆▇▇▇▇▇▇▇████████████████████████████
val_loss,█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
val_accuracy,▁▄▆▇▇███████████████████████████████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd '/content/drive/MyDrive/DL-A3 Dataset'
# %%capture
# !curl -SL https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar > dataset.tar
# !tar -xvf dataset.tar
# !rm dataset.tar
# %cd 'dakshina_dataset_v1.0/hi/lexicons'

In [34]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [35]:
%cd lexicons

[WinError 2] The system cannot find the file specified: 'lexicons'
D:\Deep Learning\cs6910-assignments\Assignment 3\lexicons


## Preparing the data

In [36]:
val_df = pd.read_csv("hi.translit.sampled.dev.tsv", sep='\t', header=None)
train_df = pd.read_csv("hi.translit.sampled.train.tsv", sep='\t', header=None)
test_df = pd.read_csv("hi.translit.sampled.test.tsv", sep='\t', header=None)

In [37]:
def process_dataframe(data):
    input_words = []
    target_words = []
    input_characters = set()
    target_characters = set()
    for x, y in zip(data[1], data[0]):
        x = str(x)
        y = str(y)
        # Add words to respective lists
        input_words.append(x)
        target_words.append("\t" + y + "\n")
        # Add characters to set
        for ch in x:
            if ch not in input_characters:
                input_characters.add(ch)
        for ch in y:
            if ch not in target_characters:
                target_characters.add(ch)
    return input_words, target_words, input_characters, target_characters

In [38]:
def get_all_words(data_list: list):
    words = set()
    for data in data_list:
        words.update(data)
    return words

In [39]:
val_input_words, val_target_words, val_input_characters, val_target_characters = process_dataframe(val_df)
train_input_words, train_target_words, train_input_characters, train_target_characters = process_dataframe(train_df)    
test_input_words, test_target_words, test_input_characters, test_target_characters = process_dataframe(test_df)    

In [40]:
input_characters = val_input_characters.union(train_input_characters, test_input_characters, [" "])
target_characters = val_target_characters.union(train_target_characters, test_target_characters, [" ","\t", "\n"])

In [41]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

all_input_words = get_all_words([val_input_words, train_input_words, test_input_words])
all_target_words = get_all_words([val_target_words, train_target_words, test_target_words])

max_encoder_seq_length = max([len(txt) for txt in all_input_words])
max_decoder_seq_length = max([len(txt) for txt in all_target_words])

In [42]:
print(f"Number of Training samples: {len(train_input_words)} | Number of Validation samples: {len(val_input_words)} | Number of Test samples: {len(test_input_words)}")
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of Training samples: 44204 | Number of Validation samples: 4358 | Number of Test samples: 4502
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21


In [43]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [44]:
def prepare_data(input_words, target_words, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length):
    encoder_input_data = np.zeros((len(input_words), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    decoder_input_data = np.zeros((len(input_words), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
    decoder_target_data = np.zeros((len(input_words), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

    for i, (input_text, target_text) in enumerate(zip(input_words, target_words)):
        # Input Data
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
        encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
        # Target Data
        for t, char in enumerate(target_text):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t, target_token_index[char]] = 1.0
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
        decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
        decoder_target_data[i, t:, target_token_index[" "]] = 1.0
        
    # Return Data
    return encoder_input_data, decoder_input_data, decoder_target_data

In [45]:
# Validation Data
val_encoder_input_data, val_decoder_input_data, val_decoder_target_data = prepare_data(val_input_words, val_target_words, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length)
# Training Data
train_encoder_input_data, train_decoder_input_data, train_decoder_target_data = prepare_data(train_input_words, train_target_words, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length)
# Test Data
test_encoder_input_data, test_decoder_input_data, test_decoder_target_data = prepare_data(test_input_words, test_target_words, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length)

## Configuration

In [46]:
batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

## Building the model

In [47]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dropout = keras.layers.Dropout(0.1)
decoder_dropout = decoder_dropout(decoder_outputs)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_dropout)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

## Training the model

In [48]:
model.compile(optimizer="nadam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()
model.fit(
    [train_encoder_input_data, train_decoder_input_data],
    train_decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([val_encoder_input_data, val_decoder_input_data], val_decoder_target_data,),
    callbacks=[WandbCallback()]
)
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model\s2s')

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, 27)]   0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None, 66)]   0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, 256), (None, 290816      input_5[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, None, 256),  330752      input_6[0][0]                    
                                                                 lstm_4[0][1]               

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file saved_model already exists.
Error occurred while processing: saved_model.


INFO:tensorflow:Assets written to: saved_model\s2s\assets


INFO:tensorflow:Assets written to: saved_model\s2s\assets


## Running inference

In [20]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
saved_model = keras.models.load_model("saved_model\s2s")
saved_model.summary()

saved_encoder_inputs = saved_model.input[0]  # input_1
saved_encoder_outputs, state_h_enc, state_c_enc = saved_model.layers[2].output  # lstm_1
saved_encoder_states = [state_h_enc, state_c_enc]
saved_encoder_model = keras.Model(saved_encoder_inputs, saved_encoder_states)

saved_decoder_inputs = saved_model.input[1]  # input_2
saved_decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
saved_decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
saved_decoder_states_inputs = [saved_decoder_state_input_h, saved_decoder_state_input_c]
saved_decoder_lstm = saved_model.layers[3]

saved_decoder_outputs, saved_state_h_dec, saved_state_c_dec = saved_decoder_lstm(saved_decoder_inputs, initial_state=saved_decoder_states_inputs)
saved_decoder_states = [saved_state_h_dec, saved_state_c_dec]
saved_decoder_dense = saved_model.layers[4]
saved_decoder_outputs = saved_decoder_dense(saved_decoder_outputs)
saved_decoder_model = keras.Model([saved_decoder_inputs] + saved_decoder_states_inputs, [saved_decoder_outputs] + saved_decoder_states)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 27)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 66)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 290816      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  330752      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [21]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [22]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = saved_encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    
    while not stop_condition:
        output_tokens, h, c = saved_decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [31]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = test_encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-"*80)
    print("Input sentence:", test_input_words[seq_index])
    print("Decoded sentence:", decoded_sentence)

--------------------------------------------------------------------------------
Input sentence: ank
Decoded sentence: अंक

--------------------------------------------------------------------------------
Input sentence: anka
Decoded sentence: अंका

--------------------------------------------------------------------------------
Input sentence: ankit
Decoded sentence: अनकित

--------------------------------------------------------------------------------
Input sentence: anakon
Decoded sentence: आंकों

--------------------------------------------------------------------------------
Input sentence: ankhon
Decoded sentence: अंखों

--------------------------------------------------------------------------------
Input sentence: ankon
Decoded sentence: अंकों

--------------------------------------------------------------------------------
Input sentence: angkor
Decoded sentence: अंगकर

--------------------------------------------------------------------------------
Input sentence: ankor
Deco